In [6]:
import pandas as pd
import numpy as np

In [7]:
# Define the path to your JSON file
csv_file_path = 'project_360_demo_remodeled.csv'

In [8]:
# Read the sample data into a DataFrame
df_sample = pd.read_csv(csv_file_path)

C:\Users\kakka\AppData\Local\Temp\ipykernel_34216\1172092947.py:2: DtypeWarning: Columns (28,29,59,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sample = pd.read_csv(csv_file_path)


In [9]:
df_sample.shape

(517211, 62)

In [10]:
import pymongo
import json
import sys

# Define your MongoDB connection parameters
host = "mongodb://localhost:27017"
database_name = "project_360_demo"

def mongo_connection(database_name, host):
    """Establishes a connection to MongoDB."""
    try:
        client = pymongo.MongoClient(host)
        db = client[database_name]  # Connects to the specified database
        print("MongoDB connection successful.")
        return db
    except Exception as e:
        print(f"Error connecting to MongoDB: {e}")
        return None

def load_event_ids(json_file):
    """Loads a list of event_ids from a JSON file."""
    try:
        with open(json_file, 'r') as file:
            data = json.load(file)
            event_ids = [item['id'] for item in data]  # Extract ids from the list of dictionaries
        print(f"Loaded {len(event_ids)} event IDs from {json_file}.")
        return event_ids
    except Exception as e:
        print(f"Error loading JSON file: {e}")
        return []

def query_events_in_batches(db, event_ids, batch_size=100000):
    """Queries MongoDB for documents in batches of event_ids."""
    if db is None:
        print("Database connection is not established.")
        return []

    collection = db['three-sixty']
    all_results = []
    total_batches = (len(event_ids) + batch_size - 1) // batch_size

    print(f"Starting query of {len(event_ids)} event IDs in {total_batches} batches.")

    for batch_num, i in enumerate(range(0, len(event_ids), batch_size), start=1):
        batch = event_ids[i:i + batch_size]
        try:
            cursor = collection.find({"event_uuid": {"$in": batch}})
            batch_results = list(cursor)  # Collect results from each batch
            all_results.extend(batch_results)
            print(f"Completed batch {batch_num} of {total_batches}.")
        except Exception as e:
            print(f"Error querying MongoDB in batch {batch_num}: {e}")
    
    print("Query completed.")
    return all_results

def save_results_to_json(results, output_file):
    """Saves the query results to a JSON file."""
    try:
        with open(output_file, 'w') as file:
            json.dump(results, file, default=str)  # default=str to handle non-serializable types
        print(f"Results saved to {output_file}.")
    except Exception as e:
        print(f"Error saving results to JSON file: {e}")

def main():
    # Load the event_ids from the JSON file
    event_ids = load_event_ids('project_360_pass_list.json')
    
    if not event_ids:
        print("No event IDs to process. Exiting.")
        sys.exit(1)
    
    # Establish connection to MongoDB
    db = mongo_connection(database_name, host)
    
    # Query the MongoDB collection in batches
    results = query_events_in_batches(db, event_ids)
    
    # Save the results to a JSON file
    save_results_to_json(results, 'data_360.json')

if __name__ == "__main__":
    main()


Loaded 517211 event IDs from project_360_pass_list.json.
MongoDB connection successful.
Starting query of 517211 event IDs in 6 batches.
Completed batch 1 of 6.
Completed batch 2 of 6.
Completed batch 3 of 6.
Completed batch 4 of 6.
Completed batch 5 of 6.
Completed batch 6 of 6.
Query completed.
Results saved to data_360.json.


In [11]:
import pandas as pd
df = pd.read_json('data_360.json')
df.shape

(38745, 5)

In [12]:
df.shape

(38745, 5)

In [13]:
event_df = pd.read_csv('project_360_demo_remodeled.csv')
event_df.head()

C:\Users\kakka\AppData\Local\Temp\ipykernel_34216\1526996059.py:1: DtypeWarning: Columns (28,29,59,61) have mixed types. Specify dtype option on import or set low_memory=False.
  event_df = pd.read_csv('project_360_demo_remodeled.csv')


,_id,id,index,period,timestamp,minute,second,type.id,type.name,possession,...,pass.through_ball,pass.technique.id,pass.technique.name,counterpress,pass.cut_back,pass.no_touch,pass.miscommunication,out,off_camera,pass.backheel
0,66ba38692d4a508f96672224,7fd797d5-530c-4e27-bac7-8fa097c95abc,28,1,00:00:16.035,0,16,30,Pass,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,66ba38692d4a508f96672228,76ebbb25-3467-413b-8497-f3e38f998e57,32,1,00:00:17.549,0,17,30,Pass,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,66ba38692d4a508f9667223d,cd460252-ecf1-4f5e-a65e-f2919175a0f2,53,1,00:00:41.641,0,41,30,Pass,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,66ba38692d4a508f9667224e,7a9e0311-6936-4f6d-aa56-90c0e6ff7b58,70,1,00:00:58.628,0,58,30,Pass,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,66ba38692d4a508f96672258,c970a836-e82c-4702-963a-b53ee7e29864,80,1,00:01:05.668,1,5,30,Pass,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
event_df.shape

(517211, 62)

In [15]:
df_merged = pd.merge(event_df,df,how='inner', left_on='id',right_on='event_uuid')
df_merged.shape

(38745, 67)

In [16]:
df_merged['pass.outcome.name'].value_counts()

Incomplete          9579
Out                 1022
Unknown              230
Pass Offside         136
Injury Clearance       5
Name: pass.outcome.name, dtype: int64

In [17]:
# Create a new column based on the condition
df_merged['class_1'] = df_merged['pass.outcome.name'].apply(
    lambda x: -1 if x in ['Incomplete', 'Out', 'Unknown', 'Pass Offside'] else 1
)

In [18]:
df_merged['class_1'].value_counts()

 1    27778
-1    10967
Name: class_1, dtype: int64

In [19]:
# Extract the first elements from the lists
pass_end_x = df_merged['pass.end_location[0]'] > df_merged['location[0]']
# Ensure we do not encounter NaNs in the comparison
df_merged['class_2'] = pass_end_x.astype(int)

In [20]:
df_merged['class_2'].value_counts()

1    22624
0    16121
Name: class_2, dtype: int64

In [21]:
# Compute the multilevel value counts using crosstab
cross_tab = pd.crosstab(df_merged['class_1'], df_merged['class_2'])

print(cross_tab)

class_2      0      1
class_1              
-1        1853   9114
 1       14268  13510


In [22]:
# Define the conditions
conditions = [
    (df_merged['class_1'] == -1),
    (df_merged['class_1'] == 1) & (df_merged['class_2'] == 0),
    (df_merged['class_1'] == 1) & (df_merged['class_2'] == 1)
]

# Define the corresponding choices
choices = [-1, 0, 1]

# Create the 'class_3' column based on the conditions
df_merged['class_3'] = np.select(conditions, choices, default=np.nan)

In [23]:
df_merged['class_3'].value_counts()

 0.0    14268
 1.0    13510
-1.0    10967
Name: class_3, dtype: int64

In [24]:
for columns in df_merged.columns:
    if 'related' in columns.lower():
        print(columns)

related_events[0]
related_events[1]
related_events[2]
related_events[3]
related_events[4]
related_events[5]


In [25]:
# Extract each column as a list
list_0 = df_merged['related_events[0]'].tolist()
list_1 = df_merged['related_events[1]'].tolist()
list_2 = df_merged['related_events[2]'].tolist()
list_3 = df_merged['related_events[3]'].tolist()
list_4 = df_merged['related_events[4]'].tolist()
list_5 = df_merged['related_events[5]'].tolist()

# Concatenate all lists into a single list
combined_list = list_0 + list_1 + list_2 + list_3 + list_4 + list_5

# Extract unique values
unique_values = list(set(combined_list))

# Output the unique values
len(unique_values)

133205

In [26]:
unique_values = list(set(combined_list))
# Output the unique values
len(unique_values)

133205

In [50]:
import pymongo
import json
import sys

# Define your MongoDB connection parameters
host = "mongodb://localhost:27017"
database_name = "project_360_demo"

def mongo_connection(database_name, host):
    """Establishes a connection to MongoDB."""
    try:
        client = pymongo.MongoClient(host)
        db = client[database_name]  # Connects to the specified database
        print("MongoDB connection successful.")
        return db
    except Exception as e:
        print(f"Error connecting to MongoDB: {e}")
        return None

def load_event_ids(json_file):
    """Loads a list of event_ids from a JSON file."""
    try:
        with open(json_file, 'r') as file:
            event_ids = json.load(file)
        print(f"Loaded {len(event_ids)} event IDs from {json_file}.")
        return event_ids
    except Exception as e:
        print(f"Error loading JSON file: {e}")
        return []

def query_events_in_batches(db, event_ids, batch_size=100000):
    """Queries MongoDB for documents in batches of event_ids with an additional check."""
    if db is None:
        print("Database connection is not established.")
        return []

    events_collection = db['events']
    three_sixty_collection = db['three-sixty']
    all_results = []
    
    # Query the 'events' collection once to get all event_ids with 'type.name': 'Pressure'
    try:
        pressure_docs = events_collection.find({"id": {"$in": event_ids}, "type.name": "Pressure"})
        pressure_ids = {doc['id'] for doc in pressure_docs}
    except Exception as e:
        print(f"Error querying events collection for Pressure type: {e}")
        return []
    
    if not pressure_ids:
        print("No events with 'type.name': 'Pressure' found.")
        return []

    total_batches = (len(pressure_ids) + batch_size - 1) // batch_size
    print(f"Starting query of {len(pressure_ids)} filtered event IDs in {total_batches} batches.")

    # Batch query the 'three-sixty' collection
    for batch_num, i in enumerate(range(0, len(pressure_ids), batch_size), start=1):
        batch = list(pressure_ids)[i:i + batch_size]
        try:
            cursor = three_sixty_collection.find({"event_uuid": {"$in": batch}})
            batch_results = list(cursor)  # Collect results from each batch
            all_results.extend(batch_results)
            print(f"Completed batch {batch_num} of {total_batches}.")
        except Exception as e:
            print(f"Error querying MongoDB in batch {batch_num}: {e}")

    print("Query completed.")
    return all_results


def save_results_to_json(results, output_file):
    """Saves the query results to a JSON file."""
    try:
        with open(output_file, 'w') as file:
            json.dump(results, file, default=str)  # default=str to handle non-serializable types
        print(f"Results saved to {output_file}.")
    except Exception as e:
        print(f"Error saving results to JSON file: {e}")

def main():
    # Load the event_ids from the JSON file
    event_ids = unique_values
    
    if not event_ids:
        print("No event IDs to process. Exiting.")
        sys.exit(1)
    
    # Establish connection to MongoDB
    db = mongo_connection(database_name, host)
    
    # Query the MongoDB collection in batches
    results = query_events_in_batches(db, event_ids)
    
    # Save the results to a JSON file
    save_results_to_json(results, 'query_result_related_events.json')

if __name__ == "__main__":
    main()


MongoDB connection successful.
Starting query of 33915 filtered event IDs in 1 batches.
Completed batch 1 of 1.
Query completed.
Results saved to query_result_related_events.json.


In [59]:
df_related = pd.read_json('query_result_related_events.json')
df_related.shape

(32016, 5)

In [60]:
related_event_col = ['related_events[1]','related_events[2]','related_events[3]','related_events[4]','related_events[5]']

In [61]:
final_df = pd.merge(df_merged,df_related,how='left', left_on='related_events[0]',right_on='event_uuid',suffixes=('','_related_events[0]'))
for col in related_event_col:
    final_df = pd.merge(final_df,df_related,how='left', left_on=col,right_on='event_uuid',suffixes=('','_'+col))

In [62]:
final_df.shape

(38745, 100)

In [63]:
for col in final_df.columns:
    if 'related' in col.lower():
        print(col)

related_events[0]
related_events[1]
related_events[2]
related_events[3]
related_events[4]
related_events[5]
event_uuid_related_events[0]
visible_area_related_events[0]
freeze_frame_related_events[0]
_id_related_events[1]
event_uuid_related_events[1]
visible_area_related_events[1]
freeze_frame_related_events[1]
match_id_related_events[1]
_id_related_events[2]
event_uuid_related_events[2]
visible_area_related_events[2]
freeze_frame_related_events[2]
match_id_related_events[2]
_id_related_events[3]
event_uuid_related_events[3]
visible_area_related_events[3]
freeze_frame_related_events[3]
match_id_related_events[3]
_id_related_events[4]
event_uuid_related_events[4]
visible_area_related_events[4]
freeze_frame_related_events[4]
match_id_related_events[4]
_id_related_events[5]
event_uuid_related_events[5]
visible_area_related_events[5]
freeze_frame_related_events[5]
match_id_related_events[5]


In [64]:
final_df.shape

(38745, 100)

In [65]:
# Drop columns where all values are NaN
df_cleaned = final_df.dropna(axis=1, how='all')

df_cleaned.shape

(38745, 93)

In [66]:
df_cleaned.columns

Index(['_id_x', 'id', 'index', 'period', 'timestamp', 'minute', 'second',
       'type.id', 'type.name', 'possession', 'possession_team.id',
       'possession_team.name', 'play_pattern.id', 'play_pattern.name',
       'team.id', 'team.name', 'player.id', 'player.name', 'position.id',
       'position.name', 'location[0]', 'location[1]', 'duration',
       'under_pressure', 'related_events[0]', 'related_events[1]',
       'related_events[2]', 'related_events[3]', 'related_events[4]',
       'pass.recipient.id', 'pass.recipient.name', 'pass.length', 'pass.angle',
       'pass.height.id', 'pass.height.name', 'pass.end_location[0]',
       'pass.end_location[1]', 'pass.body_part.id', 'pass.body_part.name',
       'match_id_x', 'pass.outcome.id', 'pass.outcome.name', 'pass.aerial_won',
       'pass.type.id', 'pass.type.name', 'pass.cross', 'pass.deflected',
       'pass.switch', 'pass.assisted_shot_id', 'pass.goal_assist',
       'pass.shot_assist', 'pass.through_ball', 'pass.technique.id'

In [70]:
# List of columns to check
columns_to_check = ['related_events[0]','related_events[1]', 'related_events[2]', 'related_events[3]','related_events[4]']

# Create a new column that counts the number of non-empty values in the specified columns
df_cleaned['non_empty_count'] = df_cleaned[columns_to_check].notna().sum(axis=1)

C:\Users\kakka\AppData\Local\Temp\ipykernel_34216\394483189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['non_empty_count'] = df_cleaned[columns_to_check].notna().sum(axis=1)


In [71]:
df_cleaned['non_empty_count'].value_counts()

2    28646
3     8902
4      803
1      391
5        3
Name: non_empty_count, dtype: int64

In [74]:
# Filter rows where 'non_empty_count' equals 1
filtered_df = df_cleaned[df_cleaned['non_empty_count'] > 1]

In [75]:
filtered_df.shape

(38354, 94)

In [76]:
filtered_df['visible_area_related_events[0]'].isna().sum()

23624

In [82]:
# Target column and columns to fill from
target_column = 'visible_area_related_events[0]'
columns_to_fill_from = ['visible_area_related_events[1]', 'visible_area_related_events[2]', 'visible_area_related_events[3]', 'visible_area_related_events[4]']

# Fill missing values in target column
filtered_df[target_column] = filtered_df[target_column].fillna(filtered_df[columns_to_fill_from].bfill(axis=1).iloc[:, 0])

C:\Users\kakka\AppData\Local\Temp\ipykernel_34216\1085627325.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[target_column] = filtered_df[target_column].fillna(filtered_df[columns_to_fill_from].bfill(axis=1).iloc[:, 0])


In [84]:
# Target column and columns to fill from
target_column = 'freeze_frame_related_events[0]'
columns_to_fill_from = ['freeze_frame_related_events[1]', 'freeze_frame_related_events[2]', 'freeze_frame_related_events[3]', 'freeze_frame_related_events[4]']

# Fill missing values in target column
filtered_df[target_column] = filtered_df[target_column].fillna(filtered_df[columns_to_fill_from].bfill(axis=1).iloc[:, 0])

C:\Users\kakka\AppData\Local\Temp\ipykernel_34216\1750192503.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[target_column] = filtered_df[target_column].fillna(filtered_df[columns_to_fill_from].bfill(axis=1).iloc[:, 0])


In [85]:
filtered_df['freeze_frame_related_events[0]'].isna().sum()

7036

In [86]:
# Target column and columns to fill from
target_column = 'event_uuid_related_events[0]'
columns_to_fill_from = ['event_uuid_related_events[1]', 'event_uuid_related_events[2]', 'event_uuid_related_events[3]', 'event_uuid_related_events[4]']

# Fill missing values in target column
filtered_df[target_column] = filtered_df[target_column].fillna(filtered_df[columns_to_fill_from].bfill(axis=1).iloc[:, 0])

C:\Users\kakka\AppData\Local\Temp\ipykernel_34216\1782317036.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[target_column] = filtered_df[target_column].fillna(filtered_df[columns_to_fill_from].bfill(axis=1).iloc[:, 0])


In [87]:
filtered_df.shape

(38354, 94)

In [88]:
filtered_df.columns

Index(['_id_x', 'id', 'index', 'period', 'timestamp', 'minute', 'second',
       'type.id', 'type.name', 'possession', 'possession_team.id',
       'possession_team.name', 'play_pattern.id', 'play_pattern.name',
       'team.id', 'team.name', 'player.id', 'player.name', 'position.id',
       'position.name', 'location[0]', 'location[1]', 'duration',
       'under_pressure', 'related_events[0]', 'related_events[1]',
       'related_events[2]', 'related_events[3]', 'related_events[4]',
       'pass.recipient.id', 'pass.recipient.name', 'pass.length', 'pass.angle',
       'pass.height.id', 'pass.height.name', 'pass.end_location[0]',
       'pass.end_location[1]', 'pass.body_part.id', 'pass.body_part.name',
       'match_id_x', 'pass.outcome.id', 'pass.outcome.name', 'pass.aerial_won',
       'pass.type.id', 'pass.type.name', 'pass.cross', 'pass.deflected',
       'pass.switch', 'pass.assisted_shot_id', 'pass.goal_assist',
       'pass.shot_assist', 'pass.through_ball', 'pass.technique.id'

In [89]:
# List of columns to remove
columns_to_remove = ['related_events[0]', 'related_events[1]','related_events[2]', 'related_events[3]', 'related_events[4]','_id_related_events[1]','event_uuid_related_events[1]', 'visible_area_related_events[1]','freeze_frame_related_events[1]', 'match_id_related_events[1]','_id_related_events[2]', 'event_uuid_related_events[2]','visible_area_related_events[2]', 'freeze_frame_related_events[2]','match_id_related_events[2]', '_id_related_events[3]','event_uuid_related_events[3]', 'visible_area_related_events[3]','freeze_frame_related_events[3]', 'match_id_related_events[3]','_id_related_events[4]', 'event_uuid_related_events[4]','visible_area_related_events[4]', 'freeze_frame_related_events[4]','match_id_related_events[4]']

# Remove the specified columns
df_master = filtered_df.drop(columns=columns_to_remove)

In [97]:
df_master['event_uuid_related_events[0]']

0        533ffee0-b9fc-4065-a9db-cb729523d926
1        c615b3b6-58de-4f77-8190-f0d7f80bbcff
2        031c2b95-22d3-4635-b1ae-c3cd3a7f2a00
3        265ff095-89ef-4223-bf0f-53bb41f3f5a6
4        2390b293-7317-40e2-90da-47c81a2f2cd5
                         ...                 
38740    77bb82d2-bdaf-46e2-abba-4ca9058ddaf3
38741    37d0ce7e-82f0-4ef1-9b1a-5231f3cb75a0
38742    db42afb8-08c2-436f-a785-16b107cc1c9f
38743    a79a45f0-bd9d-41eb-89cf-2b103217a1f0
38744    104cf7a6-4320-4a00-8c8c-aaed49d7afc8
Name: event_uuid_related_events[0], Length: 38354, dtype: object

In [91]:
df_master.head()

,_id_x,id,index,period,timestamp,minute,second,type.id,type.name,possession,...,match_id_y,class_1,class_2,class_3,_id,event_uuid_related_events[0],visible_area_related_events[0],freeze_frame_related_events[0],match_id,non_empty_count
0,66ba38692d4a508f96672224,7fd797d5-530c-4e27-bac7-8fa097c95abc,28,1,00:00:16.035,0,16,30,Pass,2,...,3895302,1,0,0.0,66ba38692d4a508f9667329b,533ffee0-b9fc-4065-a9db-cb729523d926,"[49.56201260049416, 50.80856566796838, 36.6254...","[{'teammate': True, 'actor': False, 'keeper': ...",3895302.0,2
1,66ba38692d4a508f96672228,76ebbb25-3467-413b-8497-f3e38f998e57,32,1,00:00:17.549,0,17,30,Pass,2,...,3895302,-1,1,-1.0,NaN,c615b3b6-58de-4f77-8190-f0d7f80bbcff,"[46.755233937336605, 55.71397276403634, 30.911...","[{'teammate': True, 'actor': False, 'keeper': ...",NaN,3
2,66ba38692d4a508f9667224e,7a9e0311-6936-4f6d-aa56-90c0e6ff7b58,70,1,00:00:58.628,0,58,30,Pass,3,...,3895302,-1,1,-1.0,66ba38692d4a508f966732be,031c2b95-22d3-4635-b1ae-c3cd3a7f2a00,"[0.0, 80.0, 0.0, 36.92010561266323, 26.3879996...","[{'teammate': True, 'actor': False, 'keeper': ...",3895302.0,3
3,66ba38692d4a508f96672258,c970a836-e82c-4702-963a-b53ee7e29864,80,1,00:01:05.668,1,5,30,Pass,4,...,3895302,-1,1,-1.0,66ba38692d4a508f966732c8,265ff095-89ef-4223-bf0f-53bb41f3f5a6,"[53.52389536966718, 66.00644482164927, 41.9860...","[{'teammate': True, 'actor': False, 'keeper': ...",3895302.0,3
4,66ba38692d4a508f96672275,9567c2e9-b970-4b92-b4ae-7843a912906d,109,1,00:01:52.335,1,52,30,Pass,6,...,3895302,1,1,1.0,66ba38692d4a508f966732e4,2390b293-7317-40e2-90da-47c81a2f2cd5,"[101.54205978885525, 76.4545607681595, 79.9492...","[{'teammate': True, 'actor': False, 'keeper': ...",3895302.0,3


In [92]:
df_master['class_3'].value_counts()

 0.0    14261
 1.0    13506
-1.0    10587
Name: class_3, dtype: int64

In [93]:
# Save the modified DataFrame to a CSV file
df_merged.to_csv('data_remodeled.csv', index=False)

In [96]:
df_merged.head()

,_id_x,id,index,period,timestamp,minute,second,type.id,type.name,possession,...,off_camera,pass.backheel,_id_y,event_uuid,visible_area,freeze_frame,match_id_y,class_1,class_2,class_3
0,66ba38692d4a508f96672224,7fd797d5-530c-4e27-bac7-8fa097c95abc,28,1,00:00:16.035,0,16,30,Pass,2,...,NaN,NaN,66ba38692d4a508f9667329c,7fd797d5-530c-4e27-bac7-8fa097c95abc,"[43.65320519956768, 80.0, 49.578776831182864, ...","[{'teammate': True, 'actor': False, 'keeper': ...",3895302,1,0,0.0
1,66ba38692d4a508f96672228,76ebbb25-3467-413b-8497-f3e38f998e57,32,1,00:00:17.549,0,17,30,Pass,2,...,NaN,NaN,66ba38692d4a508f966732a0,76ebbb25-3467-413b-8497-f3e38f998e57,"[48.176817444164826, 80.0, 52.193375619266625,...","[{'teammate': True, 'actor': False, 'keeper': ...",3895302,-1,1,-1.0
2,66ba38692d4a508f9667224e,7a9e0311-6936-4f6d-aa56-90c0e6ff7b58,70,1,00:00:58.628,0,58,30,Pass,3,...,NaN,NaN,66ba38692d4a508f966732bf,7a9e0311-6936-4f6d-aa56-90c0e6ff7b58,"[99.0442749628932, 80.0, 85.24408899496598, 80...","[{'teammate': False, 'actor': False, 'keeper':...",3895302,-1,1,-1.0
3,66ba38692d4a508f96672258,c970a836-e82c-4702-963a-b53ee7e29864,80,1,00:01:05.668,1,5,30,Pass,4,...,NaN,NaN,66ba38692d4a508f966732c9,c970a836-e82c-4702-963a-b53ee7e29864,"[12.955506538062423, 80.0, 38.901678469596696,...","[{'teammate': True, 'actor': False, 'keeper': ...",3895302,-1,1,-1.0
4,66ba38692d4a508f96672275,9567c2e9-b970-4b92-b4ae-7843a912906d,109,1,00:01:52.335,1,52,30,Pass,6,...,NaN,NaN,66ba38692d4a508f966732e5,9567c2e9-b970-4b92-b4ae-7843a912906d,"[0.0, 80.0, 0.0, 38.73442253628191, 21.6711782...","[{'teammate': True, 'actor': False, 'keeper': ...",3895302,1,1,1.0
